# Austrian Emissions Inventory

This notebook processes the Austrian Emissions Inventory as published by the Umweltbundesamt
for use in the **energy-scenarios-at** Scenario Explorer.

## Latest Inventory Update (2026)

### News item

https://www.umweltbundesamt.at/news260121/treibhausgas-emissionen-2024

### Report

https://www.umweltbundesamt.at/studien-reports/publikationsdetail?pub_id=2648

> Austria's Annual Greenhouse Gas Inventory 1990–2024.  
> Submission under Regulation (EU) No 2018/1999.  
> Wien, 2026  
> Reports, Band 1016  
> ISBN: 978-3-99004-863-4  
> 59 S.

### Data References

CRT file: https://www.data.gv.at/datasets/78bd7b69-c1a7-456b-8698-fac3b24f7aa5

> Umweltbundesamt GmbH, ‘Treibhausgas-Emissionen nach CRT’, 2026 (updated 2026-01-21), accessed 2026-02-07
> http://data.europa.eu/88u/dataset/78bd7b69-c1a7-456b-8698-fac3b24f7aa5

KSG file: https://www.data.gv.at/datasets/1b750209-8745-4794-93ce-1f7fc7587213

> Umweltbundesamt GmbH, ‘Treibhausgas-Emissionen nach KSG’, 2026 (updated 2026-01-21), accessed 2026-02-07,
> http://data.europa.eu/88u/dataset/1b750209-8745-4794-93ce-1f7fc7587213

Both data files released under CC-BY according to data.gv.at & data.europa.eu

In [ ]:
file_crt = "thg-emissionen_1990-2024_nach_crt_wide"
file_ksg = "thg-emissionen_1990-2024_nach_ksg_wide"

year = 2026
period = "1990-2024"
citation = "Austria's Annual Greenhouse Gas Inventory 1990–2024, Report 1016 (UBA, 2026)"
data_source = "https://www.data.gv.at/datasets/78bd7b69-c1a7-456b-8698-fac3b24f7aa5"

In [ ]:
import pandas as pd
import pyam
import nomenclature

# Emissions according to CRT

In [ ]:
emissions_crt = pd.read_csv(f"source/{file_crt}.csv", encoding="mac_roman", sep=";", decimal=",")

In [ ]:
crf = emissions_crt[["CRF_Code", "CRF_Sektor"]].drop_duplicates()
dict(
    [(row["CRF_Code"], row["CRF_Sektor"]) for i, row in crf.iterrows()]
)

In [ ]:
crf_mapping = {
    "0": " [excl. LULUCF]",
    "1 A": "|Energy",
    "1 A 1": "|Energy|Supply",
    "1 A 2": "|Energy|Demand|Industry",
    "1 A 3": "|Energy|Demand|Transportation",
    "1 A 4": "|Energy|Demand|Residential and Commercial",
    "1 A 5": "|Energy|Demand|Other",
    "1 B": "|Fugitive",
    "2": "|Industrial Processes",
    "3": "|Agriculture",
    "4": "|Land Use, Land-Use Change and Forestry",
    "5": "|Waste",
}

In [ ]:
emissions_crt.drop(columns=["Region", "Einheit", "Quelle", "CRF_Sektor", "Datenstand"], inplace=True)

In [ ]:
df_list = []

for gas, data in emissions_crt.groupby("Schadstoff"):
    if gas == "THG":
        gas = "Kyoto Gases"
        unit = "t CO2e"
    elif gas in ["CO2", "CH4", "N2O"]:
        unit = f"t {gas}"
    else:
        continue
    
    _df = pyam.IamDataFrame(
        data.drop_duplicates().drop(columns="Schadstoff"),
        model=f"Umweltbundesamt ({year})",
        scenario=f"Emission Inventory ({period})",
        region="Austria",
        variable="CRF_Code",
        unit=unit,
    )

    variable_mapping = dict(
        [(key, f"Emissions|{gas}{value or ''}") for key, value in crf_mapping.items()]    
    )
    _df = (
        _df.filter(variable=variable_mapping)
        .rename(variable=variable_mapping, check_duplicates=False)
        .convert_unit(unit, "Mt " + unit[2:])
    )
    _df.aggregate(f"Emissions|{gas}|Energy|Demand", append=True)
    _df.aggregate(
        f"Emissions|{gas}|Energy and Industrial Processes",
        components=[f"Emissions|{gas}|Energy", f"Emissions|{gas}|Industrial Processes"],
        append=True,
    )
    _df.aggregate(
        f"Emissions|{gas}",
        components=[f"Emissions|{gas} [excl. LULUCF]", f"Emissions|{gas}|Land Use, Land-Use Change and Forestry"],
        append=True,
        
    )
    
    df_list.append(_df)

df = pyam.concat(df_list)

In [ ]:
df.convert_unit("Mt N2O", "kt N2O", inplace=True)

# Emissions according to KSG

In [ ]:
emissions_ksg = pd.read_csv(f"source/{file_ksg}.csv", encoding="mac_roman", sep=";", decimal=",")

In [ ]:
emissions_ksg = emissions_ksg[emissions_ksg.Region == "÷sterreich"]

In [ ]:
emissions_ksg = emissions_ksg.drop(columns=["Region", "Schadstoff", "Einheit", "Quelle", "Datenstand"])

In [ ]:
df_ksg = pyam.IamDataFrame(
    emissions_ksg,
    model=f"Umweltbundesamt ({year})",
    scenario=f"Emission Inventory ({period})",
    region="Austria",
    variable=["Einteilung", "Sektor"],
    unit="t CO2e",
).convert_unit("t CO2e", "Mt CO2e")

In [ ]:
ksg_mapping = {
    "EH|Energie & Industrie": "Emissions|Kyoto Gases [ETS]",
    "KSG|Gesamt": "Emissions|Kyoto Gases [ESR]",
}

In [ ]:
df_ksg = df_ksg.filter(variable=ksg_mapping.keys()).rename(variable=ksg_mapping)

In [ ]:
df.append(df_ksg, inplace=True)

# Finalize and export

In [ ]:
definition = nomenclature.DataStructureDefinition("../../definitions/")

In [ ]:
definition.validate(df)

In [ ]:
df.set_meta(name="Citation", meta=citation)
df.set_meta(name="Data Source", meta=data_source)

In [ ]:
df

In [ ]:
df.to_excel(f"uba_emission_inventory_{year}.xlsx")